In [89]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')


/bin/bash: conda: command not found
/bin/bash: conda: command not found
Folium installed
Libraries imported.


I am going to use a dataset with the most crowded.

In [90]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_English_districts_by_population_density')[0]
df

,Rank,District,Density(per km²),Type of district,Ceremonial county
0,1,Islington,16097,London borough,Greater London
1,2,Tower Hamlets,16057,London borough,Greater London
2,3,Hackney,14681,London borough,Greater London
3,4,Kensington and Chelsea,12884,"London borough, Royal Borough",Greater London
4,5,Lambeth,12157,London borough,Greater London
5,6,Camden,12035,London borough,Greater London
6,7,Westminster,11883,"London borough, City (1540)",Greater London
7,8,Hammersmith and Fulham,11308,London borough,Greater London
8,9,Southwark,10992,London borough,Greater London


In [91]:
address = 'London, UK'
from geopy.geocoders import Nominatim

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [0]:
df['adress']=df['District']+','+df['Ceremonial county']
df['Latitude'] = df['adress'].apply(geolocator.geocode).apply(lambda x: (x.latitude))
df['Longitude'] = df['adress'].apply(geolocator.geocode).apply(lambda x: (x.longitude))


In [93]:
df

,Rank,District,Density(per km²),Type of district,Ceremonial county,adress,Latitude,Longitude
0,1,Islington,16097,London borough,Greater London,"Islington,Greater London",51.538429,-0.099905
1,2,Tower Hamlets,16057,London borough,Greater London,"Tower Hamlets,Greater London",51.508672,-0.016912
2,3,Hackney,14681,London borough,Greater London,"Hackney,Greater London",51.543240,-0.049362
3,4,Kensington and Chelsea,12884,"London borough, Royal Borough",Greater London,"Kensington and Chelsea,Greater London",51.498995,-0.199123
4,5,Lambeth,12157,London borough,Greater London,"Lambeth,Greater London",51.501301,-0.117287
5,6,Camden,12035,London borough,Greater London,"Camden,Greater London",51.542305,-0.139560
6,7,Westminster,11883,"London borough, City (1540)",Greater London,"Westminster,Greater London",51.501356,-0.124930
7,8,Hammersmith and Fulham,11308,London borough,Greater London,"Hammersmith and Fulham,Greater London",51.492038,-0.223640
8,9,Southwark,10992,London borough,Greater London,"Southwark,Greater London",51.502922,-0.103458


In [94]:
CLIENT_ID = 'aaaaaaaaaaaaaaa' # your Foursquare ID
CLIENT_SECRET = 'bbbbbbbbbbbbbbbbbb' # your Foursquare Secret
VERSION = '20180604'
from IPython.display import Image 
from IPython.core.display import HTML 
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=200
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    nearby_venues.head()
    return(nearby_venues)
venues = getNearbyVenues(names=df['District'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Islington
Tower Hamlets
Hackney
Kensington and Chelsea
Lambeth
Camden
Westminster
Hammersmith and Fulham
Southwark


In [95]:
venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Islington,51.538429,-0.099905,The Life Centre Islington,51.538056,-0.099171,Yoga Studio
1,Islington,51.538429,-0.099905,Steve Hatt,51.538588,-0.099041,Fish Market
2,Islington,51.538429,-0.099905,Ottolenghi,51.539716,-0.102314,Mediterranean Restaurant
3,Islington,51.538429,-0.099905,Raabs The Bakers,51.539463,-0.097768,Bakery
4,Islington,51.538429,-0.099905,Zaffrani's,51.539255,-0.101202,Indian Restaurant
...,...,...,...,...,...,...,...
603,Southwark,51.502922,-0.103458,Rose & Crown,51.505613,-0.105581,Pub
604,Southwark,51.502922,-0.103458,Bankside Cafe,51.503891,-0.106263,Breakfast Spot
605,Southwark,51.502922,-0.103458,Starbucks,51.504035,-0.109478,Coffee Shop
606,Southwark,51.502922,-0.103458,Chinese Canteen,51.500599,-0.108020,Chinese Restaurant


In [96]:
# one hot encoding
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = venues['Neighborhood'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,Afghan Restaurant,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Arts & Crafts Store,Asian Restaurant,Austrian Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Beer Store,Bistro,Boarding House,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Business Service,Butcher,Café,Canal,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Cosmetics Shop,Creperie,Cupcake Shop,...,Restaurant,Road,Rock Club,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoothie Shop,Snack Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Camden,0.000000,0.011765,0.011765,0.000000,0.000000,0.00,0.000000,0.011765,0.000000,0.011765,0.000000,0.023529,0.023529,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.035294,0.011765,0.023529,0.000000,0.000000,0.000000,0.023529,0.011765,0.082353,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.011765,0.000000,0.011765,0.000000,0.00,0.011765,0.000000,0.000000,0.000000,0.000000,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,0.023529,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023529,0.000000,0.023529,0.000000,0.000000,0.000000,0.00,0.000000
1,Hackney,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.033333,0.000000,0.000000,0.00,0.033333
2,Hammersmith and Fulham,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.010989,0.000000,0.000000,0.021978,0.010989,0.010989,0.000000,0.000000,0.010989,0.000000,0.000000,0.021978,0.000000,0.010989,0.021978,0.010989,0.010989,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.010989,0.032967,0.010989,0.065934,0.000000,0.010989,0.010989,0.010989,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.043956,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010989,0.000000,0.000000,0.000000,0.000000,0.021978,0.010989,0.000000,0.000000,0.010989,0.010989,0.000000,0.010989,0.000000,0.021978,0.010989,0.010989,0.000000,0.000000,0.000000,0.010989,0.00,0.000000
3,Islington,0.011494,0.000000,0.000000,0.000000,0.000000,0.00,0.022989,0.000000,0.011494,0.000000,0.045977,0.000000,0.000000,0.000000,0.011494,0.000000,0.011494,0.011494,0.022989,0.000000,0.000000,0.045977,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.011494,0.000000,0.011494,0.000000,0.022989,0.022989,0.011494,0.000000,0.000000,0.000000,0.000000,...,0.022989,0.011494,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011494,0.011494,0.000000,0.000000,0.000000,0.011494,0.000000,0.000000,0.000000,0.022989,0.000000,0.000000,0.000000,0.011494,0.022989,0.000000,0.011494,0.011494,0.000000,0.000000,0.000000,0.011494,0.011494,0.000000,0.000000,0.00,0.011494
4,Kensington and Chelsea,0.000000,0.00

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [0]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [0]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [100]:
venues_sorted


,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Camden,Pub,Coffee Shop,Burger Joint,Italian Restaurant,Café,Ice Cream Shop,Bar,Clothing Store,Supermarket,Beer Bar
1,Hackney,Coffee Shop,Pub,Restaurant,Café,Sporting Goods Shop,Boutique,Brewery,Shopping Plaza,Butcher,Cocktail Bar
2,Hammersmith and Fulham,Pub,Café,Coffee Shop,Sandwich Place,Hotel,Gym / Fitness Center,Clothing Store,Burger Joint,Breakfast Spot,Pizza Place
3,Islington,Pub,Bakery,Burger Joint,Café,Mediterranean Restaurant,Gastropub,Theater,Sushi Restaurant,Mexican Restaurant,Coffee Shop
4,Kensington and Chelsea,Café,Clothing Store,Burger Joint,Italian Restaurant,Cupcake Shop,Supermarket,Pub,Modern European Restaurant,Bakery,Restaurant
5,Lambeth,Hotel,Coffee Shop,Bar,Sandwich Place,Bakery,Italian Restaurant,Café,Steakhouse,Korean Restaurant,Restaurant
6,Southwark,Hotel,Pub,Coffee Shop,Gym / Fitness Center,Bar,Sandwich Place,Café,Cocktail Bar,Burger Joint,Italian Restaurant
7,Tower Hamlets,Coffee Shop,Burger Joint,Pizza Place,Bakery,Café,Steakhouse,Street Food Gathering,English Restaurant,Sushi Restaurant,Gym / Fitness Center
8,Westminster,Plaza,Pub,Historic Site,Monument / Landmark,Outdoor Sculpture,Café,Garden,Coffee Shop,Hotel,Japanese Restaurant


In [101]:
kclusters = 4

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:40]

array([0, 0, 0, 0, 3, 2, 0, 2, 1], dtype=int32)

In [106]:
london_grouped_clustering=df
london_grouped_clustering['Cluster Labels'] = kmeans.labels_
london_grouped_clustering

,Rank,District,Density(per km²),Type of district,Ceremonial county,adress,Latitude,Longitude,Cluster Labels
0,1,Islington,16097,London borough,Greater London,"Islington,Greater London",51.538429,-0.099905,0
1,2,Tower Hamlets,16057,London borough,Greater London,"Tower Hamlets,Greater London",51.508672,-0.016912,0
2,3,Hackney,14681,London borough,Greater London,"Hackney,Greater London",51.543240,-0.049362,0
3,4,Kensington and Chelsea,12884,"London borough, Royal Borough",Greater London,"Kensington and Chelsea,Greater London",51.498995,-0.199123,0
4,5,Lambeth,12157,London borough,Greater London,"Lambeth,Greater London",51.501301,-0.117287,3
5,6,Camden,12035,London borough,Greater London,"Camden,Greater London",51.542305,-0.139560,2
6,7,Westminster,11883,"London borough, City (1540)",Greater London,"Westminster,Greater London",51.501356,-0.124930,0
7,8,Hammersmith and Fulham,11308,London borough,Greater London,"Hammersmith and Fulham,Greater London",51.492038,-0.223640,2
8,9,Southwark,10992,London borough,Greater London,"Southwark,Greater London",51.502922,-0.103458,1


In [0]:
london_grouped_clustering=london_grouped_clustering.drop(columns=['Rank','adress','Ceremonial county'])
london_grouped_clustering=london_grouped_clustering.rename(columns={'District':'Street'})

In [108]:
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')
london_grouped_clustering 


,Street,Density(per km²),Type of district,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Islington,16097,London borough,51.538429,-0.099905,0,Pub,Bakery,Burger Joint,Café,Mediterranean Restaurant,Gastropub,Theater,Sushi Restaurant,Mexican Restaurant,Coffee Shop
1,Tower Hamlets,16057,London borough,51.508672,-0.016912,0,Coffee Shop,Burger Joint,Pizza Place,Bakery,Café,Steakhouse,Street Food Gathering,English Restaurant,Sushi Restaurant,Gym / Fitness Center
2,Hackney,14681,London borough,51.543240,-0.049362,0,Coffee Shop,Pub,Restaurant,Café,Sporting Goods Shop,Boutique,Brewery,Shopping Plaza,Butcher,Cocktail Bar
3,Kensington and Chelsea,12884,"London borough, Royal Borough",51.498995,-0.199123,0,Café,Clothing Store,Burger Joint,Italian Restaurant,Cupcake Shop,Supermarket,Pub,Modern European Restaurant,Bakery,Restaurant
4,Lambeth,12157,London borough,51.501301,-0.117287,3,Hotel,Coffee Shop,Bar,Sandwich Place,Bakery,Italian Restaurant,Café,Steakhouse,Korean Restaurant,Restaurant
5,Camden,12035,London borough,51.542305,-0.139560,2,Pub,Coffee Shop,Burger Joint,Italian Restaurant,Café,Ice Cream Shop,Bar,Clothing Store,Supermarket,Beer Bar
6,Westminster,11883,"London borough, City (1540)",51.501356,-0.124930,0,Plaza,Pub,Historic Site,Monument / Landmark,Outdoor Sculpture,Café,Garden,Coffee Shop,Hotel,Japanese Restaurant
7,Hammersmith and Fulham,11308,London borough,51.492038,-0.223640,2,Pub,Café,Coffee Shop,Sandwich Place,Hotel,Gym / Fitness Center,Clothing Store,Burger Joint,Breakfast Spot,Pizza Place
8,Southwark,10992,London borough,51.502922,-0.103458,1,Hotel,Pub,Coffee Shop,Gym / Fitness Center,Bar,Sandwich Place,Café,Cocktail Bar,Burger Joint,Italian Restaurant


In [111]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clusters)
       
map_clusters

In [0]:
cluster1=london_grouped_clustering[london_grouped_clustering['Cluster Labels']==0] 
cluster2=london_grouped_clustering[london_grouped_clustering['Cluster Labels']==1] 
cluster3=london_grouped_clustering[london_grouped_clustering['Cluster Labels']==2] 
cluster4=london_grouped_clustering[london_grouped_clustering['Cluster Labels']==3] 



In [113]:
cluster1    

,Street,Density(per km²),Type of district,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Islington,16097,London borough,51.538429,-0.099905,0,Pub,Bakery,Burger Joint,Café,Mediterranean Restaurant,Gastropub,Theater,Sushi Restaurant,Mexican Restaurant,Coffee Shop
1,Tower Hamlets,16057,London borough,51.508672,-0.016912,0,Coffee Shop,Burger Joint,Pizza Place,Bakery,Café,Steakhouse,Street Food Gathering,English Restaurant,Sushi Restaurant,Gym / Fitness Center
2,Hackney,14681,London borough,51.543240,-0.049362,0,Coffee Shop,Pub,Restaurant,Café,Sporting Goods Shop,Boutique,Brewery,Shopping Plaza,Butcher,Cocktail Bar
3,Kensington and Chelsea,12884,"London borough, Royal Borough",51.498995,-0.199123,0,Café,Clothing Store,Burger Joint,Italian Restaurant,Cupcake Shop,Supermarket,Pub,Modern European Restaurant,Bakery,Restaurant
6,Westminster,11883,"London borough, City (1540)",51.501356,-0.124930,0,Plaza,Pub,Historic Site,Monument / Landmark,Outdoor Sculpture,Café,Garden,Coffee Shop,Hotel,Japanese Restaurant


In [114]:
cluster2

,Street,Density(per km²),Type of district,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Southwark,10992,London borough,51.502922,-0.103458,1,Hotel,Pub,Coffee Shop,Gym / Fitness Center,Bar,Sandwich Place,Café,Cocktail Bar,Burger Joint,Italian Restaurant


In [115]:
cluster3

,Street,Density(per km²),Type of district,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Camden,12035,London borough,51.542305,-0.13956,2,Pub,Coffee Shop,Burger Joint,Italian Restaurant,Café,Ice Cream Shop,Bar,Clothing Store,Supermarket,Beer Bar
7,Hammersmith and Fulham,11308,London borough,51.492038,-0.22364,2,Pub,Café,Coffee Shop,Sandwich Place,Hotel,Gym / Fitness Center,Clothing Store,Burger Joint,Breakfast Spot,Pizza Place


In [116]:
cluster4

,Street,Density(per km²),Type of district,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lambeth,12157,London borough,51.501301,-0.117287,3,Hotel,Coffee Shop,Bar,Sandwich Place,Bakery,Italian Restaurant,Café,Steakhouse,Korean Restaurant,Restaurant
